In [1]:
# import subprocess
# import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

In [2]:
from cfg import *
from util import *

##### main
data = json.load(open(Config.data_path+"train.json"))
data_pd=pd.DataFrame(data)
len(data_pd[data_pd['tokens'].apply(len)>2024])

30

In [3]:
Config.modelsavepath

'/home/z1933/workplace/machinelearning/PII-Data-Detection//modelsave/'

In [4]:
data_pd.iloc[data_pd.full_text.str.len().idxmax()]

document                                                           21720
full_text              In this assignment, a reflective report will b...
tokens                 [In, this, assignment, ,, a, reflective, repor...
trailing_whitespace    [True, True, False, True, True, True, True, Tr...
labels                 [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
Name: 6365, dtype: object

In [5]:
tokenizer = AutoTokenizer.from_pretrained(Config.model_name)
tokenized_input =tokenizer(data_pd.iloc[data_pd.full_text.str.len().idxmax()]['full_text'],return_tensors='pt')

/home/z1933/miniconda3/envs/d2l/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [6]:
# tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
len(tokenized_input["input_ids"])
tokenized_input["input_ids"].size()

torch.Size([1, 2900])

In [7]:
tokenized_input.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [8]:
from itertools import chain
#将所有数据的labels连接在一起,然后查重,转成list的格式,然后从小到大排序
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
print(f"all_labels:{all_labels}")
label2id = {l: i for i,l in enumerate(all_labels)}
#这个是{id:label},上面是{label:id}
id2label = {v:k for k,v in label2id.items()}

# Open a file for writing
with open(Config.modelsavepath+"//idlabel.json", "w") as f:
    # Write the map to the file in JSON format
    json.dump({'label2id':label2id,'id2label':id2label}, f)
id2label

all_labels:['B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL', 'O']


{0: 'B-EMAIL',
 1: 'B-ID_NUM',
 2: 'B-NAME_STUDENT',
 3: 'B-PHONE_NUM',
 4: 'B-STREET_ADDRESS',
 5: 'B-URL_PERSONAL',
 6: 'B-USERNAME',
 7: 'I-ID_NUM',
 8: 'I-NAME_STUDENT',
 9: 'I-PHONE_NUM',
 10: 'I-STREET_ADDRESS',
 11: 'I-URL_PERSONAL',
 12: 'O'}

========================================================================================================================
Layer (type:depth-idx)                                                 Output Shape              Param #
========================================================================================================================
NLPModel                                                               [1, 100, 13]              --
├─DebertaV2ForTokenClassification: 1-1                                 [1, 100, 13]              --
│    └─DebertaV2Model: 2-1                                             [1, 100, 1024]            --
│    │    └─DebertaV2Embeddings: 3-1                                   [1, 100, 1024]            131,176,448
│    │    └─DebertaV2Encoder: 3-2                                      [1, 100, 1024]            302,835,712
│    └─Dropout: 2-2                                                    [1, 100, 1024]            --
│    └─Linear: 2-3                                                     [1, 100, 13]              13,325
========================================================================================================================
Total params: 434,025,485
Trainable params: 434,025,485
Non-trainable params: 0
Total mult-adds (M): 484.93
========================================================================================================================
Input size (MB): 0.00
Forward/backward pass size (MB): 423.44
Params size (MB): 1734.00
Estimated Total Size (MB): 2157.45
=======================================================================================================================

## EDA

In [9]:
# EDA
tranjs_path="dataset/train.json"
train_df = json.load(open(tranjs_path))
print(f"len(train_df):{len(train_df)}, train_df[0].keys(): {list(train_df[0].keys())}")
print("-"*50)


print(f"train_df length:", len(train_df))

labels = set()
label_counts = {}
for i in range(len(train_df)):
    labels.update(train_df[i]['labels'])
    for label in train_df[i]['labels']:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1
            
print(f"labels: {labels}")
print('-'*25)
print(f"label_counts: {label_counts}")

len(train_df):6807, train_df[0].keys(): ['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels']
--------------------------------------------------
train_df length: 6807
labels: {'B-USERNAME', 'I-NAME_STUDENT', 'I-STREET_ADDRESS', 'I-ID_NUM', 'I-URL_PERSONAL', 'B-URL_PERSONAL', 'O', 'B-ID_NUM', 'B-STREET_ADDRESS', 'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-EMAIL', 'I-PHONE_NUM'}
-------------------------
label_counts: {'O': 4989794, 'B-NAME_STUDENT': 1365, 'I-NAME_STUDENT': 1096, 'B-URL_PERSONAL': 110, 'B-EMAIL': 39, 'B-ID_NUM': 78, 'I-URL_PERSONAL': 1, 'B-USERNAME': 6, 'B-PHONE_NUM': 6, 'I-PHONE_NUM': 15, 'B-STREET_ADDRESS': 2, 'I-STREET_ADDRESS': 20, 'I-ID_NUM': 1}


In [10]:

# # EDA
# # display bar
# import copy
# display_label=copy.copy(label_counts)
# del  display_label['O']
# y =  [i for i in display_label.values()]

# g=sns.barplot(x=list(display_label.keys()),y=y)
# g.set_xticklabels(g.get_xticklabels(), rotation=90)

In [11]:
tmp_list=[]
for i in train_df:
    for j in range(0,len(i['tokens'])):
        if i['tokens'][j].isspace() and i['labels'][j]!='O':
            tmp_list.append([i['tokens'][j-1],i['labels'][j-1],i['tokens'][j],i['labels'][j],i['tokens'][j+1],i['labels'][j+1]])
tmp_list

[['656',
  'I-STREET_ADDRESS',
  '\n',
  'I-STREET_ADDRESS',
  'Joshuamouth',
  'I-STREET_ADDRESS'],
 ['419',
  'I-STREET_ADDRESS',
  '\n',
  'I-STREET_ADDRESS',
  'Andreahaven',
  'I-STREET_ADDRESS']]

In [12]:
df_train=pd.DataFrame(train_df)
df_train_tokens=[]
for row in df_train.itertuples(index=False):
    for i in range(len(row.tokens)):
        df_train_tokens.append((row.document,row.tokens[i],row.labels[i]))
df_train_tokens=pd.DataFrame(df_train_tokens,columns=['document','tokens','labels'])
df_train_tokens

,document,tokens,labels
0,7,Design,O
1,7,Thinking,O
2,7,for,O
3,7,innovation,O
4,7,reflexion,O
...,...,...,...
4992528,22687,process,O
4992529,22687,explained,O
4992530,22687,above,O
4992531,22687,.,O


In [13]:
# inspect specific label
inspect="PHONE"
df_temp=df_train_tokens.loc[ df_train_tokens.labels.str.contains(inspect)]
print(df_temp.to_string())

        document        tokens       labels
150909      4381             (  B-PHONE_NUM
150910      4381       320)202  I-PHONE_NUM
150911      4381             -  I-PHONE_NUM
150912      4381    0688x95843  I-PHONE_NUM
179880      4777             (  B-PHONE_NUM
179881      4777       223)392  I-PHONE_NUM
179882      4777             -  I-PHONE_NUM
179883      4777          2765  I-PHONE_NUM
286835      6243             (  B-PHONE_NUM
286836      6243       820)913  I-PHONE_NUM
286837      6243             -  I-PHONE_NUM
286838      6243      3241x894  I-PHONE_NUM
287266      6243             (  B-PHONE_NUM
287267      6243       820)913  I-PHONE_NUM
287268      6243             -  I-PHONE_NUM
287269      6243      3241x894  I-PHONE_NUM
287293      6243             (  B-PHONE_NUM
287294      6243       820)913  I-PHONE_NUM
287295      6243             -  I-PHONE_NUM
287296      6243      3241x894  I-PHONE_NUM
861215      9854  410.526.1667  B-PHONE_NUM


In [14]:
input_data=pd.DataFrame(data)

In [15]:
input_data.loc[input_data['document']==4381]

,document,full_text,tokens,trailing_whitespace,labels
185,4381,"WRITING CENTRE Level 3 East, Hub Central Nor...","[WRITING, CENTRE, , Level, 3, East, ,, Hub, C...","[True, True, False, True, True, False, True, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [16]:
# DEA
# print all unicode character
# def has_unicode_codepoints(text):
#     return any(ord(char) > 127 for char in text)
# tmp_set=set()
# pattern =re.compile(r"[^\u0000-\u00ff]")
# for i in data:
#     for j in i['tokens']:
#         if has_unicode_codepoints(j):
#             tmp_set.add(j)
# for i in tmp_set:
#     print(i)

In [17]:
# !cp ./nltk_data/* ~/nltk_data/ -r

In [18]:
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [x["document"] for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
    "labels": [x["labels"] for x in data]
})

预测时，空缺值填'O'

In [19]:
# EDA
# import matplotlib.pyplot as plt
# import seaborn as sns
# tmp_list=[]
# for doc in data:
#     full_text=doc['full_text']
#     split_lines=full_text.split('\n\n')
#     for i in split_lines:
#         tmp_list.append(len(tokenizer(i)['input_ids']))
# print(len(tmp_list))
# print(max(tmp_list))
# sns.displot(tmp_list)

In [20]:
# 如果多次没击中，大概率有问题
preprocesssed_ds=ds.map(preprocesss, fn_kwargs={'tokenizer':tokenizer,'label2id':label2id,'if_train':True},num_proc=8,desc="prepocessing data")
preprocesssed_ds

prepocessing data (num_proc=8):   0%|          | 0/6807 [00:00<?, ? examples/s]

Dataset({
    features: ['full_text', 'document', 'tokens', 'trailing_whitespace', 'labels', 'splited_sens', 'berttokenpos2orgtokenpos', 'bertlabels', 'berttokenids', 'berttokenmask', 'berttokentoken_type_ids'],
    num_rows: 6807
})

In [21]:
assert len(preprocesssed_ds[0]['berttokenids']) == len(preprocesssed_ds[0]['splited_sens'])

In [22]:
tmp_str="."
tmp_res=tokenizer(tmp_str)
tmp_res

{'input_ids': [1, 323, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [23]:
tokenizer.convert_ids_to_tokens(tmp_res['input_ids'])

['[CLS]', '▁.', '[SEP]']

In [24]:
# rebuid dataset 
tmp_pd=expanddataset(preprocesssed_ds)
tmp_pd['berttokenids'].str.len().agg(['mean','max','std'])

mean      44.367558
max     1036.000000
std       73.714812
Name: berttokenids, dtype: float64

In [25]:
full_ds=Dataset.from_pandas(tmp_pd)
full_ds

Dataset({
    features: ['full_text', 'document', 'tokens', 'trailing_whitespace', 'labels', 'splited_sens', 'berttokenpos2orgtokenpos', 'berttokenids', 'berttokenmask', 'berttokentoken_type_ids', 'bertlabels'],
    num_rows: 110943
})

In [26]:
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader,random_split
from transformers import DataCollatorForTokenClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_dataset, val_dataset = random_split(full_ds, [0.8, 0.2],generator=torch.Generator().manual_seed(Config.seed)) #TODO:改变比例


In [27]:
print(len(train_dataset))

88755


In [29]:
data_collator = Collate(tokenizer=tokenizer)
train_dataloader=DataLoader(train_dataset,batch_size=Config.batch_size,pin_memory=True,collate_fn=data_collator,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=Config.batch_size,pin_memory=True,collate_fn=data_collator)

In [30]:
from model1 import NLPModel
model=NLPModel(id2label,label2id).to(device)
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
import gc

optimizer= AdamW(model.parameters(),lr=Config.lr,weight_decay=Config.weight_decay)

scaler = torch.cuda.amp.GradScaler()
valstep = len(train_dataloader) //Config.evaltimes

num_train_steps = int(len(train_dataloader) * Config.epochs)
scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=Config.num_warmup_steps, num_training_steps=num_train_steps, 
            )

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
def compute_metrics(predictions,traget, all_labels):
    
    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, traget)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, traget)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results

def validation(val_dataloader,model):
    model.eval()

    allprecitions=[]
    alltargets=[]
    for step,dataset in enumerate(tqdm(val_dataloader)):

        with torch.no_grad():
            ids = dataset["ids"].to(device,non_blocking=True)
            mask = dataset["mask"].to(device,non_blocking=True)
            targets = dataset["targets"].to(device,non_blocking=True)
            tokentype = dataset["type_ids"].to(device,non_blocking=True)
            logit,loss = model(ids,mask, tokentype,targets)

            targets=targets.view(-1)
            targets = targets.detach().cpu().numpy()

            allprecitions.append(np.argmax(logit, axis=1))
            alltargets.append(targets)
        del ids,mask,targets,tokentype,loss,logit

    score=compute_metrics(allprecitions,alltargets,all_labels)
    
    gc.collect()
    torch.cuda.empty_cache()

    model.train()

    return score

In [31]:
gc.collect()
torch.cuda.empty_cache()
start_time=time.time()
bestscore=0
epoch=0

if Config.resume_train_epoch:
    checkpoint = torch.load(Config.workdir+f"//checkpoint//checkpoint{Config.resume_train_epoch}.pth")
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    epoch = checkpoint['epoch']
    bestscore=checkpoint['bestscore']
    model.train()
    print(f"start epoch:{epoch}")

while epoch <Config.epochs:
    losses=[] 
    for step,dataset in enumerate(tqdm(train_dataloader)):
        with torch.cuda.amp.autocast():
            ids = dataset["ids"].to(device,non_blocking=True)
            mask = dataset["mask"].to(device,non_blocking=True)
            targets = dataset["targets"].to(device,non_blocking=True)
            tokentype = dataset["type_ids"].to(device,non_blocking=True)

            logit,loss = model(ids,mask, tokentype,targets)
            loss/= Config.accumulation_steps

        # Accumulates scaled gradients.
        scaler.scale(loss).backward()
        # losses.append(loss.item())
        
        if (step+1) % Config.accumulation_steps ==0:
            scaler.step(optimizer)
            scale = scaler.get_scale()
            # Updates the scale for next iteration.
            scaler.update()
            skip_lr_sched = (scale != scaler.get_scale())
            if not skip_lr_sched:
                scheduler.step()
            optimizer.zero_grad()

        # validation
        if epoch>0 and (step+1) % valstep == 0:
            scores=validation(val_dataloader,model)
            print("Validation:Epoch{} Step [{}/{}] Loss: {:.3f} Time: {:.1f} Metric: {}".format(epoch,step, len(train_dataloader)-1, loss.item(), time.time()-start_time,scores))
            if scores['f1']>bestscore:
                print(f"Best score is {bestscore} → {scores['f1']}. Saving model")
                torch.save(model.state_dict(), os.path.join(Config.modelsavepath,"model_seed{}_score{:.3f}.pth".format(Config.seed,bestscore)))
                bestscore=scores['f1']
            else:
                print("no improvement, bestf1 is {:.3f} score is {}".format(bestscore,scores))
                # report loss
        if (step+1) % Config.logging_steps ==0:
            # print("Logging: Step [{}/{}] Loss: {:.3f} Time: {:.1f}".format(step, len(train_dataloader)-1, loss.item(), time.time()-start_time))
            gc.collect()
            torch.cuda.empty_cache()
        del ids,mask,targets,tokentype,loss,logit

    epoch+=1
    # checkpoint
    torch.save({
            'epoch': epoch,
            'bestscore':bestscore,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict':scheduler.state_dict(),
            'scaler_state_dict':scaler.state_dict()
            }, Config.workdir+"//checkpoint/checkpoint{}.pth".format(epoch))

os.system("/usr/bin/shutdown")

  0%|          | 67/22189 [00:10<1:00:21,  6.11it/s]


KeyboardInterrupt: 